## Context 

raw features capture the full predictive signal in the data. FE transfroms domain knowledge into model inputs. creating new perspectives on the data that can imporve model performance significanly

## objective 

- create time-derived features from TransactionDT
- build card-level based on address level aggregation features
- engginer email domain and interaction features
-  document transformation logic for profuction reproducibility

In [1]:
# import and load data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

train = pd.read_parquet(Path('../data/interim/train_merged.parquet'))
print(f'Data loaded: {train.shape}')

Data loaded: (590540, 434)


In [3]:
# time features 

START_DATE = pd.Timestamp('2017-11-30')
train['TransactionDate'] = START_DATE + pd.to_timedelta(train['TransactionDT'], unit='s')

train['hour'] = train['TransactionDate'].dt.hour
train['day_of_week'] = train['TransactionDate'].dt.dayofweek
train['day_of_month'] = train['TransactionDate'].dt.day
train['is_night'] = train['hour'].apply(lambda x : 1 if x >= 22 or x <= 5 else 0)
train['is_weekend'] = train['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

print('Time features created : ')
print(train[['hour', 'day_of_week', 'is_night', 'is_weekend']].describe().transpose())

Time features created : 
                count       mean       std  min  25%   50%   75%   max
hour         590540.0  13.861923  7.607152  0.0  6.0  16.0  20.0  23.0
day_of_week  590540.0   2.928123  1.947733  0.0  1.0   3.0   5.0   6.0
is_night     590540.0   0.378897  0.485113  0.0  0.0   0.0   1.0   1.0
is_weekend   590540.0   0.254101  0.435355  0.0  0.0   0.0   1.0   1.0


## insight : time based feature engginering

from the result we have created these essential time features:
- hour : hour of day 00-23. fraud rates peak during night hours
- day_of_week : 0= monday to 6 = sunday. weekend patterns differ from weekdays
- is_night : binary flag for 10 P.M to 5 A.M transaction when fraud is elevated
- is_weekend : binary flag for saturday and sunday transactions

these simple features consistenly rank in the top 50 by importance in tree models, providing high value for low engginering cost